<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/dev/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Config and Requirements
!pip install coolname gdown papermill json5
from IPython.display import clear_output
from coolname import generate_slug
from os.path import join
from shlex import quote
clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

---Settings---
IPFS_ROOT: /content/ipfs
IPFS_PEER: ipfs.pollinations.ai


In [2]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!mkdir -p $worker_root


!echo "Created IPFS resource" > $ipfs_root/output/log
clear_output()

In [3]:
#@title Install Pollinations CLI

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev 
!git pull
!npm run install_backend
%cd -

#clear_output()

/content
Cloning into 'pollinations'...
remote: Enumerating objects: 2846, done.
remote: Counting objects: 100% (429/429), done.
remote: Compressing objects: 100% (250/250), done.
remote: Total 2846 (delta 318), reused 249 (delta 174), pack-reused 2417
Receiving objects: 100% (2846/2846), 19.48 MiB | 29.50 MiB/s, done.
Resolving deltas: 100% (1927/1927), done.
/content/pollinations/app
Already up to date.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
Already up to date.

> pollinations@0.4.5 install_backend /content/pollinations/app
> cp -v dist/pollinate.js /usr/local/bin/pollinate

'dist/pollinate.js' -> '/usr/local/bin/pollinate'
/content


In [4]:
#@title Run IPFS daemon &

!/content/pollinations/ipfs/ipfsd_install_colab.sh
!nohup  /content/pollinations/ipfs/ipfsd_forever.sh >> $ipfs_root/output/log &
!sleep 5
clear_output()


In [5]:
#@title Determine node ID

from os.path import join


node_id=!ipfs id -f="<id>\n"
node_id=node_id[0]

print("---Node ID---:", node_id)

---Node ID---: 12D3KooWL3bU6b29Zq2BHyKqZzWavnAfg7pz3igoUwhJ54pWTNom


In [6]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))




<IPython.core.display.Javascript object>

In [7]:
#@title Receive first IPFS state. Watch and send ipfs output folder &
!DEBUG=* pollinate -p $ipfs_root --ipns --receive --once
!python /content/pollinations/pollinations/prepare_for_papermill.py $notebook_path
#!nohup pollinate -p $ipfs_root --ipns --receive &
!nohup pollinate -p $ipfs_root --ipns --receive &
!nohup pollinate -p $ipfs_root --ipns --send &

clear_output()

In [8]:
#@title Notebook Runner

from glob import glob
from os.path import basename
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

output_path = ipfs_root+"/output"

first_run=True

def get_params():
  parameters={}
  for path in glob(f"{ipfs_root}/input/*"):
    key = basename(path)
    with open(path,'r') as f: 
      value = f.read()
      parameters[key] = loads(value)
  return parameters

def run(parameters):
  print("Running",parameters)
 
  parameters["output_path"] = output_path
  _params = [["-p", quote(key), quote(str(value))] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]


  cmd = ["papermill", notebook_path, "/content/notebook_output.ipynb"] + params + ["--log-output"]
  print("Running cmd"," ".join(cmd))
  #command = " ".join(cmd)
  #!$cmd
  #return
  with open(ipfs_root+"/output/log","a") as logout:
    Popen(cmd, 
            stdout = STDOUT, 
            stderr = STDOUT, 
            shell = False, 
            cwd = worker_root
            ).communicate()# as p:
    print("popen",p)
      
   
        
while True:
  if not first_run:
    !DEBUG=* pollinate -p $ipfs_root --ipns --receive --once
    !python /content/pollinations/pollinations/prepare_for_papermill.py $notebook_path
  
  first_run = False
  parameters = get_params()
  print("New parameters", parameters)
  run(parameters)
  !sleep 5
  break

New parameters {'text_not': 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing', 'num_iterations': 700, 'text_input': 'Vibrant painting of me in the style of Dali. '}
Running {'text_not': 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing', 'num_iterations': 700, 'text_input': 'Vibrant painting of me in the style of Dali. '}
Running cmd papermill /content/ipfs/notebook.ipynb /content/notebook_output.ipynb -p text_not 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing' -p num_iterations 700 -p text_input 'Vibrant painting of me in the style of Dali. ' -p output_path /content/ipfs/output --log-output


OSError: ignored